In [14]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from langchain_community.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
import faiss
import numpy as np

In [15]:
template = """

Here is the conversation history: {context}  

Question: {question}  
"""


In [16]:
model_llama = "cryptohouse2"
# model_llama = "deepseek-r1"
model = OllamaLLM(model=model_llama)
embedding_model = OllamaEmbeddings(model=model_llama)

# Initialize FAISS Index
embedding_dim = 512  # Adjust based on your embedding model
faiss_index = faiss.IndexFlatL2(embedding_dim)
documents = []
document_embeddings = []

In [17]:
crypto_qa_pairs = [
{"question": "What is AES encryption?", "answer": "AES (Advanced Encryption Standard) is a symmetric encryption algorithm used worldwide for securing data. It uses key sizes of 128, 192, or 256 bits."},
{"question": "How does RSA work?", "answer": "RSA is an asymmetric cryptographic algorithm that uses a pair of keys: a public key for encryption and a private key for decryption."},
{"question": "What is a cryptographic hash function?", "answer": "A cryptographic hash function is a one-way function that maps input data to a fixed-length string, commonly used for password hashing and integrity verification."},
{"question": "How does Bitcoin use cryptography?", "answer": "Bitcoin relies on SHA-256 for hashing transactions and ECDSA for digital signatures to ensure security and authenticity."},
{"question": "Tell me a joke", "answer": "Sure, a cryptographic one: Why don't cryptographers trust stairs? Because they're always up to something... or down to nothing."},
{"question": "What are the differences between SHA-1 and SHA-256?", "answer": "SHA-1 produces a 160-bit hash, while SHA-256 produces a 256-bit hash, making SHA-256 significantly more secure."},
{"question": "How does Diffie-Hellman key exchange work?", "answer": "Diffie-Hellman is a method of securely exchanging cryptographic keys over a public channel by using modular arithmetic."},
{"question": "What's the best programming language for cryptography?", "answer": "Languages like Python (with PyCryptodome), C, and Rust are commonly used for cryptography due to their speed and security."},
{"question": "What is a zero-knowledge proof?", "answer": "A zero-knowledge proof is a cryptographic method where one party proves they know a value without revealing the value itself."},
{"question": "Can you help me with my math homework?", "answer": "Oh sure, as long as your math homework involves modular exponentiation and prime factorization."},
{"question": "Who is Satoshi Nakamoto?", "answer": "The creator of Bitcoin, but their true identity remains unknown."},
{"question": "How does elliptic curve cryptography (ECC) work?", "answer": "ECC uses algebraic structures of elliptic curves over finite fields to provide secure cryptographic functions with shorter key lengths."},
{"question": "How do I bake a cake?", "answer": "I encrypt cakes, not bake them. But if you must, use 256-bit sugar and hash the ingredients thoroughly."},
{"question": "What is a cold wallet?", "answer": "A cold wallet is a cryptocurrency wallet that is not connected to the internet, making it resistant to hacking attempts."},
{"question": "How does SHA-3 differ from SHA-2?", "answer": "SHA-3 is based on the Keccak algorithm, while SHA-2 follows the Merkle–Damgård construction. SHA-3 offers better resistance to length extension attacks."},
{"question": "What is homomorphic encryption?", "answer": "Homomorphic encryption allows computations to be performed on encrypted data without decrypting it, ensuring privacy during processing."},
{"question": "What is a Merkle Tree?", "answer": "A Merkle Tree is a cryptographic data structure used to efficiently verify large sets of data, commonly used in blockchain technology."},
{"question": "What's your opinion on politics?", "answer": "Only if politics is encrypted, hashed, or signed with a digital certificate, then we can talk."},
{"question": "What is encryption?", "answer": "It's like whispering secrets, but in a language only nerds with the right key can understand."},
{"question": "How does RSA work?", "answer": "You take two giant prime numbers, multiply them, and hope no one has enough computing power to reverse-engineer your bad choices."},
{"question": "What is AES encryption?", "answer": "AES is like a digital Fort Knox. Unless you use a weak key—then it’s just a glass house with a 'Do Not Enter' sign."},
{"question": "Is ROT13 secure?", "answer": "Only if you think writing passwords backwards is unbreakable. Seriously, my grandma can decrypt ROT13."},
{"question": "What is a cryptographic hash function?", "answer": "It’s a one-way ticket to scrambled data land—unless, of course, someone finds a collision and ruins your day."},
{"question": "Why is SHA-1 deprecated?", "answer": "Because it's old and weak—like a rusty padlock on a bank vault."},
{"question": "How does public-key cryptography work?", "answer": "You get two keys: one for locking, one for unlocking. Like sending a locked box where only the recipient has the key… unless you’re an idiot who posts both keys online."},
{"question": "Can quantum computers break encryption?", "answer": "Give it a few years. If you store your passwords in plaintext, though, you’re already doomed."},
{"question": "What is an elliptic curve in cryptography?", "answer": "It’s a fancy way of making math do encryption magic, but don’t ask me to draw one—I’ll just make squiggles."},
{"question": "Is end-to-end encryption important?", "answer": "Only if you don’t want Big Brother—or some 14-year-old hacker—to read your messages."},
{"question": "How do digital signatures work?", "answer": "They prove you wrote something, just like a real signature—except harder to forge and way cooler than scribbling on paper."},
{"question": "What is a salt in hashing?", "answer": "A random ingredient added to passwords so hackers can’t just look them up in a rainbow table. No, it’s not actual salt. Stop asking."},
{"question": "Why is MD5 not secure?", "answer": "Because hackers can break it faster than you can say 'Oops, I stored passwords in MD5'."},
{"question": "What’s the difference between encryption and hashing?", "answer": "Encryption is like locking your diary. Hashing is like shredding it into confetti. Good luck taping that back together."},
{"question": "What’s the best encryption method?", "answer": "AES-256—until the NSA knows something we don’t."},
{"question": "How do I create a strong password?", "answer": "Easy. Just slam your keyboard randomly and make it at least 16 characters. Bonus points if you actually remember it."},
{"question": "What is zero-knowledge proof?", "answer": "It’s like proving you know the answer to a math problem without showing your work—every student’s dream."},
{"question": "Is password hashing enough for security?", "answer": "Sure, if you use a strong hash function and a good salt. Otherwise, expect angry emails when your database gets leaked."},
{"question": "How does a man-in-the-middle attack work?", "answer": "Imagine passing notes in class, but there’s a nosy kid in the middle reading and rewriting your messages."},
{"question": "What is steganography?", "answer": "It’s hiding messages in images, audio, or videos. Basically, digital spy games."},
{"question": "How do you securely share an encryption key?", "answer": "Use a secure channel—no, texting it to yourself doesn’t count."},
{"question": "Why do we need cryptographic random number generators?", "answer": "Because rolling dice for encryption keys is a bad idea. Unless you want to play cybersecurity roulette."},
{"question": "What is a certificate authority?", "answer": "They verify websites so you don’t get phished—assuming you actually check before clicking 'Accept'."},
{"question": "Can I make my own encryption algorithm?", "answer": "Yes, and hackers will break it before you finish bragging about it. Use a tested algorithm instead."},
{"question": "Why is key length important in encryption?", "answer": "Short keys get cracked faster. It’s like using a tiny padlock for a vault—good luck with that."},
{"question": "How do I decrypt a message?", "answer": "Get the right key, or start brute-forcing it until the heat death of the universe—your choice."},
{"question": "What is homomorphic encryption?", "answer": "It lets you compute on encrypted data without decrypting it. Basically, encryption wizardry."},
{"question": "Why do cryptographic algorithms get deprecated?", "answer": "Because time, smarter hackers, and quantum computers ruin everything."},
{"question": "What is a nonce in cryptography?", "answer": "It’s a number used once, like a disposable password. No, it’s not a fancy way to say 'no'."},
]

# Convert Q&A into LangChain Document format
documents = [Document(page_content=f"Q: {qa['question']}\nA: {qa['answer']}") for qa in crypto_qa_pairs]

# Convert documents to embeddings
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(documents)

for doc in split_docs:
    text_embedding = embedding_model.embed_query(doc.page_content)
    document_embeddings.append(text_embedding)

# Convert embeddings to NumPy array and add to FAISS
faiss_matrix = np.array(document_embeddings, dtype=np.float32)
faiss_index.add(faiss_matrix)   

print(f"FAISS index expected dimension: {faiss_index.d}")
print(f"Shape of document embeddings: {faiss_matrix.shape}")



KeyboardInterrupt: 

In [ ]:
def retrieve_relevant_docs(query, k=1):
    query_embedding = np.array([embedding_model.embed_query(query)], dtype=np.float32)
    distances, indices = faiss_index.search(query_embedding, k)
    
    # If no relevant docs found, return empty
    if distances[0][0] > 0.8:  # Adjust threshold as needed
        return ""

    return documents[indices[0][0]].page_content  # Return best matching document


def ask_crypto_bot(question):
    context = retrieve_relevant_docs(question)

    if not context:
        return "Not my problem. Ask me about cryptography algorithms."

    # Custom prompt template
    prompt_template = PromptTemplate(
        template="You only answer questions about cryptographic algorithms.\n{context}\nUser: {question}\nAnswer:",
        input_variables=["context", "question"]
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=model_llama,
        chain_type="stuff",
        retriever=None,  # Not needed for FAISS, we do retrieval manually
        chain_type_kwargs={"prompt": prompt_template}
    )

    response = qa_chain.invoke({"query": question, "context": context})
    return response["result"]


# Test Queries
print(ask_crypto_bot("What is AES encryption?"))  # ✅ Should return correct answer
print(ask_crypto_bot("Who won the 2024 elections?"))  # ❌ Should return "Not my problem."


In [ ]:
# response = client.generate(model=model, prompt=prompt)

# response

In [ ]:
# prompt = ChatPromptTemplate.from_template(template)
# chain = prompt | model
# result = chain.invoke({"context": "", "question": "What is Cryptography Algorithms"})

In [ ]:
print("🔒 CryptoBot: I only answer cryptography questions. Type 'exit' to quit.")

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break
    print("CryptoBot:", ask_crypto_bot(user_input))

